
## Import 

In [ ]:
cd ..

In [ ]:
pwd

In [ ]:
import os, sys, pickle

from IPython.display import display

import requests

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()
import plotly.express as px

from sklearn.model_selection import * 
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.impute import * 
from sklearn.preprocessing import * 
from sklearn.ensemble import * 
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.dummy import DummyClassifier

from category_encoders import TargetEncoder

# import shap

## Load data

In [ ]:
cleaned = "./data/cleaned/"

In [ ]:
df = pd.read_csv(cleaned+"df.csv")

In [ ]:
df.head()

## Prepare

In [ ]:
df.head()

In [ ]:
X = df.drop(columns="Survived")
y = df.Survived

In [ ]:
df_train, df_test  = train_test_split(df)

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
y_train = df_train.Survived.values

In [ ]:
# X_train_PassengerId = df_train.PassengerId.values

X_train = df_train.drop(columns=["Survived", ], inplace=False)
X_train.head()

In [ ]:
y_train[:10]

In [ ]:
len(y_train)

In [ ]:
len(X_train)

In [ ]:
y_test = df_test.Survived.values


In [ ]:
# X_test_PassengerId = df_test.PassengerId.values
X_test = df_test.drop(columns=["Survived" ], inplace=False)
X_test.head()

In [ ]:
X_train.select_dtypes(include=np.number).columns

In [ ]:
X_train.select_dtypes(exclude=np.number).columns

## Model

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ("num", "passthrough",['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']),
    ("oneHot", OneHotEncoder( handle_unknown="ignore"), ['Sex', 'Embarked'])
    ], remainder="drop")

In [ ]:
X_train.head(10)

In [ ]:
tmp = preprocessor.fit_transform(X_train)
pd.DataFrame(tmp).head(10)

In [ ]:
imputer = KNNImputer()

# X_train = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns)
# X_test = pd.DataFrame(imputer.transform(X_test), columns=X_train.columns)
# X_train.info()

In [ ]:
pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("imputer",imputer),
    ("estimator", LGBMClassifier())
])
pipe

In [ ]:
param_grid = {
    "estimator" : [DummyClassifier(), LGBMClassifier()]
    }


In [ ]:
grid = GridSearchCV(pipe, 
                    param_grid, 
                    cv=StratifiedShuffleSplit(n_splits=10, test_size=0.33), 
                    return_train_score=True, 
                    n_jobs=-1,
                    verbose=1)

## Train

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
res = pd.DataFrame(grid.cv_results_)
cols = [i for i in res.columns if ("split" not in i) ]
res = res.loc[:, cols]
res.sort_values("mean_test_score", ascending=False).round(2)

## Test

In [ ]:
grid.score(X_train, y_train)

In [ ]:
model = grid.best_estimator_
model

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
pre = grid.best_estimator_['preprocessor']
pre


In [ ]:
imp = grid.best_estimator_['imputer']
imp

In [ ]:
est = grid.best_estimator_['estimator']
est

# export

In [ ]:
with open("./models/model.pk", "wb") as f : 
    pk = pickle.dumps(model)
    f.write(pk)

In [ ]:
with open("./models/preprocessor.pk", "wb") as f : 
    pk = pickle.dumps(pre)
    f.write(pk)

In [ ]:
with open("./models/imputer.pk", "wb") as f : 
    pk = pickle.dumps(imp)
    f.write(pk)

In [ ]:
with open("./models/estimator.pk", "wb") as f : 
    pk = pickle.dumps(est)
    f.write(pk)

In [ ]:
display(len(X_train))
# display(len(X_train_PassengerId))
display(len(y_train))

In [ ]:
len(y_train)

In [ ]:
X_train["Survived"] = y_train
# X_train["PassengerId"] = X_train_PassengerId
X_train.head()

In [ ]:
X_train.isna().sum()

In [ ]:
X_train.to_csv("./data/cleaned/df_train.csv", index=False)

In [ ]:
X_test["Survived"] = y_test
# X_test["PassengerId"] = X_test_PassengerId
X_test.head()

In [ ]:
X_test.isna().sum()

In [ ]:
X_test.to_csv("./data/cleaned/df_test.csv", index=False)